Импорт необходимых библиотек

In [1]:

from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split # Импортируем функцию train_test_split для разделения данных на обучающий и тестовый наборы
from scipy.stats import mstats
from sklearn.preprocessing import StandardScaler
import pandas as pd #Импортируем pandas для удобства работы с данными
import pickle #Импортируем библиотеку, которая сохраняет модель
import numpy as np

Открытие датасетов

In [8]:
df_train = pd.read_csv("C://Users//yegor//Desktop//datatech_internship//model//X_data.csv",sep = ";").rename(columns = {"Unnamed: 0":"Date"})
y_train = pd.read_csv("C://Users//yegor//Desktop//datatech_internship//model//Y_train.csv", sep=';', names=['Время забора пробы', 'Проба'])

Работа с датасетами

In [9]:
outlier_columns = ['T_data_1_1', 'T_data_1_2', 'T_data_1_3', 'T_data_2_1', 'T_data_2_2', 'T_data_2_3',
'T_data_3_1', 'T_data_3_2', 'T_data_3_3', 'T_data_4_1', 'T_data_4_2', 'T_data_4_3',
'T_data_5_1', 'T_data_5_2', 'T_data_5_3', 'AH_data']

for col in outlier_columns:
    df_train[col] = mstats.winsorize(df_train[col], limits=[0.05, 0.05])

df_train['Date'] = pd.to_datetime(df_train['Date'], format='%d.%m.%Y %H:%M')
y_train['Время забора пробы'] = pd.to_datetime(y_train['Время забора пробы'], format='%d.%m.%Y %H:%M')


# Объединение по условию совпадения дат
merged_data = pd.merge(df_train, y_train, left_on='Date', right_on='Время забора пробы', how='inner')
merged_data.drop("Время забора пробы",axis = 1,inplace = True)
merged_data['Date'] = pd.to_datetime(merged_data['Date'], format='%d.%m.%Y %H:%M')

# Создание новых признаков на основе колонки 'Date'
merged_data['Year'] = merged_data['Date'].dt.year
merged_data['Month'] = merged_data['Date'].dt.month
merged_data['Day'] = merged_data['Date'].dt.day
merged_data['Hour'] = merged_data['Date'].dt.hour
merged_data['Minute'] = merged_data['Date'].dt.minute
merged_data.drop("Date",axis = 1,inplace = True)



Разбиение переменных

In [10]:
X = merged_data.drop(['Проба'], axis=1)
y = merged_data['Проба']


X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

scaler = StandardScaler()

# Применение StandardScaler к обучающему набору данных (fit_transform)
X_train = scaler.fit_transform(X)

# Применение StandardScaler к тестовому набору данных (transform)
X_test = scaler.transform(X_test)


Открытие обученной модели

In [11]:
with open('C://Users//yegor//Desktop//datatech_internship//model//lr_reg.pkl', 'rb') as file:
    model = pickle.load(file)

Реализация PFI

**Permutation Feature Importance** — это метод оценки важности признаков в моделях машинного обучения. Он основан на идее, что если признак важен для предсказания, то его случайное перемешивание (перемешивание значений) должно значительно ухудшить производительность модели


1. **Обучение модели**:
- Сначала обучается модель на исходном наборе данных и вычисляется её производительность (например, точность, F1-меры и т.д.) на валидационном наборе.
2. **Перемешивание признаков**:
- Для каждого признака в наборе данных:
- Признак перемешивается (перемешиваются значения этого признака в выборке), что разрушает связь между этим признаком и целевой переменной.

3. **Вычисление важности**:
- Важность признака определяется как разница между производительностью модели до и после перемешивания:
$$
\text{Importance} = \text{Performance}_{\text{original}} - \text{Performance}_{\text{permuted}}
$$


 - \(\text{Performance}_{\text{original}}\): Это метрика производительности модели (например, точность, F1-меры, AUC и т.д.), рассчитанная на исходном наборе данных, где все признаки находятся на своих местах. Это значение служит базовой линией для оценки важности признаков.
- **\(\text{Performance}_{\text{permuted}}\)**: Это метрика производительности модели, рассчитанная на наборе данных, в котором значения одного или нескольких признаков были случайным образом перемешаны (перемешаны).


- Чем больше разница, тем более важен признак для модели.
5. **Повторение**:
- Процесс повторяется несколько раз (обычно 10-30 раз) для получения более стабильной оценки важности, и затем вычисляется среднее значение и стандартное отклонение важности для каждого признака.

In [12]:
result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
importances = result.importances_mean
std_errors = result.importances_std

# Сортируем важности в убывающем порядке
sorted_indices = np.argsort(importances)[::-1]

# Выводим важности признаков
feature_names = X.columns
for f in range(X.shape[1]):
    print(f"{feature_names[sorted_indices[f]]}: {importances[sorted_indices[f]]:.3f} (+/- {std_errors[sorted_indices[f]]:.3f})")

T_data_3_1: 0.219 (+/- 0.007)
T_data_3_3: 0.178 (+/- 0.005)
T_data_3_2: 0.175 (+/- 0.005)
H_data: 0.052 (+/- 0.002)
T_data_5_3: 0.020 (+/- 0.001)
T_data_5_2: 0.012 (+/- 0.001)
T_data_5_1: 0.012 (+/- 0.001)
T_data_1_3: 0.010 (+/- 0.001)
T_data_1_2: 0.009 (+/- 0.000)
T_data_1_1: 0.006 (+/- 0.000)
T_data_2_1: 0.004 (+/- 0.000)
T_data_2_3: 0.004 (+/- 0.000)
T_data_2_2: 0.002 (+/- 0.000)
T_data_4_3: 0.000 (+/- 0.000)
Minute: 0.000 (+/- 0.000)
AH_data: -0.000 (+/- 0.000)
T_data_4_2: -0.000 (+/- 0.000)
Day: -0.000 (+/- 0.000)
Hour: -0.000 (+/- 0.000)
T_data_4_1: -0.000 (+/- 0.000)
Year: -0.000 (+/- 0.000)
Month: -0.000 (+/- 0.000)




#### Результаты анализа

После выполнения кода были получены следующие результаты важности признаков (среднее значение и стандартная ошибка):

1. **T_data_3_1**: 0.219 (+/- 0.007)
2. **T_data_3_3**: 0.178 (+/- 0.005)
3. **T_data_3_2**: 0.175 (+/- 0.005)
4. **H_data**: 0.052 (+/- 0.002)
5. **T_data_5_3**: 0.020 (+/- 0.001)
6. **T_data_5_2**: 0.012 (+/- 0.001)
7. **T_data_5_1**: 0.012 (+/- 0.001)
8. **T_data_1_3**: 0.010 (+/- 0.001)
9. **T_data_1_2**: 0.009 (+/- 0.000)
10. **T_data_1_1**: 0.006 (+/- 0.000)
11. **T_data_2_1**: 0.004 (+/- 0.000)
12. **T_data_2_3**: 0.004 (+/- 0.000)
13. **T_data_2_2**: 0.002 (+/- 0.000)
14. **T_data_4_3**: 0.000 (+/- 0.000)
15. **Minute**: 0.000 (+/- 0.000)
16. **AH_data**: -0.000 (+/- 0.000)
17. **T_data_4_2**: -0.000 (+/- 0.000)
18. **Day**: -0.000 (+/- 0.000)
19. **Hour**: -0.000 (+/- 0.000)
20. **T_data_4_1**: -0.000 (+/- 0.000)
21. **Year**: -0.000 (+/- 0.000)
22. **Month**: -0.000 (+/- 0.000)

#### Обсуждение результатов

Из результатов видно, что температурные показатели в камерах (T_data_3_1, T_data_3_3, T_data_3_2) оказывают наибольшее влияние на предсказательную способность модели. Это может указывать на то, что температура является ключевым фактором в процессе, который моделируется.

Высота слоя (H_data) также имеет значительную важность, что подчеркивает её роль в процессе. Влажность сырья (AH_data) и временные признаки (Minute, Day, Hour, Year, Month) показывают незначительное или отсутствующее влияние на модель, что может свидетельствовать о их меньшей роли в контексте рассматриваемого процесса.

